In [1]:
# ---- load globals ----
source("R/globals.R", local = TRUE)

source("R/data_loading.R", local = TRUE)

source("R/helpers-line.R") 
source("R/module_rotate.R", local = TRUE)

library(plotly)       
library(base64enc)    

library(SPARK)       # sparkx() 在这里
library(Matrix)
library(DT)
library(promises)
library(future)
plan(multisession)
library(DESpace)
library(SpatialExperiment)
library(SummarizedExperiment)
library(dplyr)
library(DT)
library(ggplot2)
source("R/module_select.R") 

source("R/download.R", local = TRUE)


source("R/plot.R", local = TRUE)

source("R/distance.R", local = TRUE)

source("R/gam_analysis.R", local = TRUE)

source("R/GO_utils.R", local = TRUE)

source("R/traj_utils.R", local = TRUE)

source("R/sparkx.R", local = TRUE)

source("R/umap.R", local = TRUE)

source("R/DeSpace.R", local = TRUE)




Attaching package: ‘jsonlite’


The following object is masked from ‘package:shiny’:

    validate



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout



Attaching package: ‘DT’


The following objects are masked from ‘package:shiny’:

    dataTableOutput, renderDataTable


Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStat

In [2]:
ui <- fluidPage(
  titlePanel("Visium HE overlay + gene expression"),
  sidebarLayout(
    sidebarPanel(
      fileInput(
        "mtx_triplet", "Upload matrix.mtx + barcodes.tsv + features.tsv",
        multiple = TRUE,
        accept = c(".gz")
      ),

      fileInput("positions_file","Upload tissue_positions_list.csv"),
      fileInput("scalef_file",   "Upload scalefactors_json.json"),
      fileInput("image_file",    "Upload HE image (png/jpg)"),

      # ---- Gene / appearance settings ----
      selectizeInput(
        "gene", "Select gene:",
        choices  = character(0),
        selected = NULL,
        options  = list(placeholder = "Type to search gene symbol...")
      ),
      checkboxInput("log1p", "log1p transform", value = TRUE),
      sliderInput("ptsize", "Point size", min = 1, max = 6, value = 3, step = 0.5),
      sliderInput("alpha", "Point transparency", min = 0.1, max = 1, value = 0.9, step = 0.1),
      helpText("Please upload the 10x matrix trio (matrix/barcodes/features) first, then pick a gene."),

      tags$hr(),
      rotateImageUI("rot"),

      # 在合适的位置（比如 sidebarPanel 里）调用模块 UI：
      sparkxUI("sparkx"),

      umapUI("umap")

    ),

    mainPanel(
      # ================= Main plot & selection =================
      radioButtons(
        "sel_mode", "Selection tool",
        choices = c("Lasso" = "lasso", "Rectangle" = "select"),
        inline = TRUE, selected = "lasso"
      ),
      plotly::plotlyOutput("he_plotly", height = "750px", width = "100%"),

      # ================= Structure annotation =================
      hr(), h4("Structure annotation (biological structure)"),
      fluidRow(
        column(
          4,
          selectizeInput(
            "bio_label", "Biological structure",
            choices  = c("tumor","tls","stroma"),
            selected = "tumor",
            options  = list(create = TRUE, persist = TRUE)   # 允许键入新增
          )
        ),
        column(4, actionButton("add_annotation", "Annotate current selection")),
        column(4, actionButton("clear_annotations", "Clear all annotations"))
      ),
      tableOutput("anno_summary"),

      # ================= A→B line tools =================
      hr(), lineUI("line"),

      
      hr(),
      svgLineUI("line"),


      # ================= Export & tools =================
      exportToolsUI_global(),



      hr(),
      trajUI("traj"),


      hr(),
      goUI("go"),

      hr(),
      despace_ui("despace1"),


      hr(),
      verbatimTextOutput("dbg")
    )
  )
)


In [3]:
# app.R（或 server.R）
server <- function(input, output, session) {

  # ====== 全局状态（集中在一个 rv）======
  rv <- reactiveValues(
    counts=NULL, gene_names=NULL, barcodes=NULL,
    pos=NULL, hires_scale=NULL,
    img=NULL, img_w=NULL, img_h=NULL, img_path=NULL,
    sparkx_res=NULL
  )

  # ====== I/O: 三件套合并上传 ======
  observeEvent(input$mtx_triplet, {
    df <- input$mtx_triplet
    req(is.data.frame(df), nrow(df) >= 3)
    paths <- pick_mtx_triplet_paths(df)     # <- helpers-io.R
    res   <- load_counts(paths$mtx, paths$bar, paths$fea)
    rv$counts     <- res$counts
    rv$gene_names <- res$gene_names
    rv$barcodes   <- res$barcodes
    updateSelectizeInput(session, "gene",
      choices = rv$gene_names, selected = character(0), server = TRUE
    )
    showNotification("Loaded matrix/barcodes/features.")

    rv$counts_spe    <- methods::as(rv$counts, "dgCMatrix")
    rownames(rv$counts_spe) <- rv$gene_names
    colnames(rv$counts_spe) <- rv$barcodes
  })


  observeEvent(input$positions_file, {
    rv$pos <- load_and_align_positions(input$positions_file$datapath, rv$counts)  # helpers-io.R
    rv$spe <- make_spe_from_matrix(rv$counts_spe, pos_df = rv$pos, sample = "sample1")
  })


  observeEvent(input$scalef_file, {
    rv$hires_scale <- load_scalef(input$scalef_file$datapath)
    if (!is.null(rv$pos)) rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
  })

  observeEvent(input$image_file, {
    im <- load_image(input$image_file$datapath)
    rv$img <- im$img; rv$img_w <- im$img_w; rv$img_h <- im$img_h; rv$img_path <- input$image_file$datapath
  })

  # ====== 表达向量与底图 ======
  expr_vec <- make_expr_vec(reactive(input$gene), reactive(rv$counts), reactive(rv$pos), reactive(isTRUE(input$log1p)))

  rot <- rotateImageServer("rot", image_path_reactive = reactive(rv$img_path))   # module-rotate-image.R
  current_img_obj <- reactive(get_current_image(rot, rv))                        # helpers-expr.R or helpers-utils.R

  # ====== 选择/标注/画线 模块 ======
  selected_idx <- reactiveVal(integer(0))
  setup_selection(output, session, selected_idx, reactive(input$sel_mode), reactive(input$clear_sel))   # module-selection.R
  annotations <- init_annotations()                                                                             # module-annotations.R
  setup_annotation_observers(input, output, rv, selected_idx, annotations)                               # module-annotations.R
  line_state <- lineModuleServer("line", rv = rv, img_obj = current_img_obj, expr_vec = expr_vec)               # module-line.R

  # ====== 主图 ======
  output$he_plotly <- renderPlotly({
    req(rv$pos, current_img_obj())
    build_he_plotly_figure(rv$pos, expr_vec(), current_img_obj(), input$ptsize, input$alpha, input$gene) %>%
      plotly::layout(
        dragmode    = sel_dragmode(input$sel_mode),            # helpers-plot.R
        shapes      = line_state$shapes(),                     # from module-line
        annotations = line_state$annotations()                 # from module-line
      )
  })

  label_index_list <- reactive(make_label_index_list(rv, annotations))
  multi_distance_df <- setup_multi_distance(output, rv, expr_vec, label_index_list)

  # ====== 沿线表达曲线 / 导出 ======
  lineCurveServer("line", rv = rv, expr_vec = expr_vec, active_line = line_state$active_line)              # module-line.R

  # ====== 1D SVG（沿线） ======
  svgLineServer("line", rv = rv, active_line = line_state$active_line)                                         # module-svg-line.R

  # ====== SPARK-X ======
  sparkxServer("sparkx", rv = rv)                                                                               # module-sparkx.R

  despace_server("despace1", rv, annotations)
  # ====== GAM 多结构 & 导出 ======
  gammmServer("gam", rv = rv, multi_distance_df_r = multi_distance_df, 
          log1p_r = reactive(input$log1p))          
  
  gam_res <- gammmServer(
  "gam",
  rv = rv,
  multi_distance_df_r = multi_distance_df,       # ⚠️ 传 reactive，本身不加 ()
  log1p_r = reactive(input$log1p)                # 如果你前面已经写了这个
)
                                                       # module-gam.R

  # ====== 轨迹（上传距离CSV、LOESS、表/图/下载） ======
  trajServer("traj", rv = rv, expr_vec = expr_vec)                                                              # module-traj.R




  # ====== GO 富集 ======
  goServer("go")                                                                               # module-go.R


  umapServer("umap", rv = rv, selected_idx_r = selected_idx)


  # # ====== Debug ======
  # output$dbg <- renderPrint(debug_dump(rv))                                                                     # helpers-utils.R
}


In [4]:
shinyApp(ui, server)


Listening on http://127.0.0.1:3954

Warning message:
“The 'plotly_selected' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_click' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_click')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_click' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_click')` to the plot (`p`) that you wish to obtain event data from.”
'as(<dgTMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").

Warning message:
“UNRELIABLE VALUE: Future (‘<none>’) unexpectedly generated random numbers without specifying argument 's